# <center>**Smartdeploy**</center>

[!] No ejecutar este notebook, ya que contiene modulos que aun no estan soportados en esta version de Smartdeploy.

# **Modulo Checker**

In [3]:
import pandas as pd
import sys
import os
import rich
from IPython.display import HTML

filename = "artifacts/X_inference.csv"
X_inference = pd.read_csv(filename)
X_inference.head()

specimen_number  eccentricity  aspect_ratio  elongation  solidity  \
0                1       0.86224        2.0735     0.52269   0.98686   
1               11       0.52382        1.1117     0.67175   0.54701   
2                3       0.82866        1.9848     0.50917   0.94180   
3                4       0.70668        1.2510     0.38111   0.94226   
4                4       0.73935        1.5319     0.34987   0.98479   

   stochastic_convexity  isoperimetric_factor  maximal_indentation_depth  \
0               0.99474               0.70529                   0.010097   
1               0.62982               0.15157                   0.136740   
2               0.99825               0.55942                   0.025524   
3               0.99825               0.69250                   0.019432   
4               1.00000               0.81067                   0.007808   

   lobedness  average_intensity  average_contrast  smoothness  third_moment  \
0   0.018554           0.041404          0.121630    0.014579      0.004869   
1   3.402800           0.026434          0.085792    0.007306      0.002137   
2   0.118570           0.080103          0.166920    0.027107      0.008655   
3   0.068724           0.031587          0.115020    0.013056      0.005311   
4   0.011095           0.027888          0.114720    0.012990      0.006017   

   uniformity  entropy  
0    0.000276  0.94580  
1    0.000166  0.90513  
2    0.000427  1.80380  
3    0.000086  0.72247  
4    0.000050  0.59895

In [4]:
sys.path.append('/home/jovyan/')
from smartdeploy.cl import checker

## Inferir tipo de input
- Tabular: pandas, numpy
- Image: numpy

In [4]:
input_type = checker.infer_input(X_inference)
print(f"Inferred input type: {input_type}")
assert input_type == 'tabular'

Inferred input type: tabular


## **Checker de Integridad** 
- Tabular: pandas, numpy
- Image: numpy

In [11]:
results = checker.integrity(X_inference, save_mlflow=False)
# results = checker.integrity(df, data_type=input_type)
df = pd.DataFrame(results)
HTML(df.to_html())

/opt/conda/lib/python3.8/site-packages/deepchecks/tabular/utils/task_inference.py:66: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



| Check Name | Description | Result |
| --- | --- | --- |
| Is Single Value | Check if there are columns which have only a single unique value in all rows. | Passed |
| Special Characters | Search in column[s] for values that contains only special characters. | Passed |
| Mixed Nulls | Search for various types of null values, including string representations of null. | Passed |
| Mixed Data Types | Detect columns which contain a mix of numerical and string values. | Passed |
| String Mismatch | Detect different variants of string categories (e.g. "mislabeled" vs "mis-labeled") in a categorical column. | Passed |
| Data Duplicates | Checks for duplicate samples in the dataset. | Passed |
| String Length Out Of Bounds | Detect strings with length that is much longer/shorter than the identified "normal" string lengths. | Passed |
| **Total Passed Tests** | **Overall result** | **100%** |


| Key | Value |
| --- | --- |
| Model UUID | 6b5489a5a7a445089b399908e068c83e |
| Run ID | 4640c7ebf5e94669a0bee12daec7e7d9 |
| Artifact Path | extratree |
| Creation Time (UTC) | 2023-04-10 22:25:59.983110 |
| MLflow Version | 1.30.0 |
| Environment | conda.yaml |
| Loader Module | mlflow.sklearn |
| Model Path | model.pkl |
| Prediction Function | predict |
| Python Version | 3.8.13 |
| Code | null |
| Pickled Model | model.pkl |
| Serialization Format | cloudpickle |
| Scikit-learn Version | 1.1.2 |
| Inputs | eccentricity, aspect_ratio, elongation, solidity, stochastic_convexity, isoperimetric_factor, etc
| Outputs | tensor of type int64 with unknown dimensions (-1) |


## ¿ Pasa todos los tests de integridad ?

In [12]:
all(df["pass"])

True

## **Checker de Drift Train vs Test**

In [13]:
X_inference = X_inference.loc[:, X_inference.columns != 'specimen_number'].copy()
X_train = pd.read_csv("../training/artifacts/X_train.csv")
X_train = X_train.loc[:, X_train.columns != 'specimen_number'].copy()

In [14]:
results = checker.drift(X_inference, X_train)
rich.print(results)

/opt/conda/lib/python3.8/site-packages/deepchecks/tabular/utils/task_inference.py:66: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



{
    'domain_classifier_auc': 0.5493827160493827,
    'domain_classifier_drift_score': 0.09876543209876543,
    'domain_classifier_feature_importance': {
        'entropy': 1.0,
        'eccentricity': 0.0,
        'aspect_ratio': -0.0,
        'elongation': 0.0,
        'solidity': 0.0,
        'stochastic_convexity': -0.0,
        'isoperimetric_factor': 0.0,
        'maximal_indentation_depth': 0.0,
        'lobedness': 0.0,
        'average_intensity': 0.0,
        'average_contrast': 0.0,
        'smoothness': 0.0,
        'third_moment': 0.0,
        'uniformity': 0.0
    }
}

## ¿ Pasa el test de drift ?

In [13]:
threshold = 0.2
results["domain_classifier_drift_score"] < threshold

True

# **Tracker**

In [6]:
from smartdeploy.cl import tracker

last_artifact = tracker.get_last_artifact(path="extratree")
last_artifact

[19:50:36] Found on this date: Mon Jul 31 19:50:28 2023                                               ]8;id=680589;file:///home/jovyan/smartdeploy/cl/tracker.py\tracker.py]8;;\:]8;id=749554;file:///home/jovyan/smartdeploy/cl/tracker.py#26\26]8;;\

{'run_uuid': '300d9fa71c64473abd5e1089220c7e34',
 'artifact_path': 's3://mlflow/0/300d9fa71c64473abd5e1089220c7e34/artifacts/extratree'}

In [7]:
import mlflow

client = mlflow.MlflowClient()

client.list_artifacts(last_artifact["run_uuid"], 'extratree')

[<FileInfo: file_size=1232, is_dir=False, path='extratree/MLmodel'>,
 <FileInfo: file_size=212, is_dir=False, path='extratree/conda.yaml'>,
 <FileInfo: file_size=4594714, is_dir=False, path='extratree/model.pkl'>,
 <FileInfo: file_size=122, is_dir=False, path='extratree/python_env.yaml'>,
 <FileInfo: file_size=95, is_dir=False, path='extratree/requirements.txt'>]

In [9]:
# client.download_artifacts(last_artifact["run_uuid"],
#                           path='X_train.csv')